In [1]:
import cobra
import logging
import cobrakbase
from cobrakbase.core.converters import KBaseFBAModelToCobraBuilder
logger = logging.getLogger(__name__)

cobrakbase 0.2.7


In [2]:
kbase = cobrakbase.KBaseAPI()

### Fetch data & info from KBase

In [3]:
ws = "filipeliu:narrative_1561151919387"
fbamodel_json = kbase.get_object('GCF_000005845.2.gf.mdl', ws)
print(fbamodel_json['id'])
print(fbamodel_json['name'])

GCF_000005845.2.mdl
Escherichia coli str. K-12 substr. MG1655


In [4]:
from cobrakbase.kbase_object_info import KBaseObjectInfo
info = KBaseObjectInfo(kbase.ws_client.get_object_info3({
    'objects' : [
        {'name' : 'GCF_000005845.2.gf.mdl', 'workspace': ws}
    ]})['infos'][0])

### Build model from data & info using the FBAModelBuilder
The builder returns a `cobrakbase.core.kbasefba.FBAModel` which inherits both `cobra.core.Model` and `cobrakbase.core.kbaseobject.KBaseObject`

In [5]:
from cobrakbase.core.kbasefba.fbamodel_builder import FBAModelBuilder
from cobrakbase.core.kbasefba import FBAModel
from cobrakbase.core.kbaseobject import KBaseObject
from cobra.core import Model
b = FBAModelBuilder.from_kbase_json(fbamodel_json, info)
model = b.build()
print(type(model))
print(isinstance(model, FBAModel))
print(isinstance(model, Model))
print(isinstance(model, KBaseObject))
print(FBAModel.mro())
model

<class 'cobrakbase.core.kbasefba.fbamodel.FBAModel'>
True
True
True
[<class 'cobrakbase.core.kbasefba.fbamodel.FBAModel'>, <class 'cobrakbase.core.kbaseobject.KBaseObject'>, <class 'cobra.core.model.Model'>, <class 'cobra.core.object.Object'>, <class 'object'>]


Name,GCF_000005845.2.mdl
Memory address,0x07feda4efc6d0
Number of metabolites,1287
Number of reactions,1566
Number of groups,0
Objective expression,1.0*bio1 - 1.0*bio1_reverse_b18f7
Compartments,"c0, e0"


### Run default FBA with solution of 298.498 (complete media)

In [6]:
model.summary()

### Get media object from KBase 
cobrakbase.core.kbasebiochem.media.Media

In [7]:
media = kbase.get_from_ws('Carbon-D-Glucose', ws)
print(type(media))

<class 'cobrakbase.core.kbasebiochem.media.Media'>


### Run FBA with Carbon-D-Glucose media
`FBAModel.medium` = **dict or Media**

- `FBAModel.medium` inherits default cobra medium setter but also accepts `cobrakbase.core.kbasebiochem.media.Media`

In [8]:
model.medium = media
model.summary()

### Run previous steps without using the builder
`get_from_ws` runs object factory it **always** return an instance of `KBaseObject`

In [9]:
model = kbase.get_from_ws('GCF_000005845.2.gf.mdl', ws)
model

Name,GCF_000005845.2.mdl
Memory address,0x07fed889bdb90
Number of metabolites,1287
Number of reactions,1566
Number of groups,0
Objective expression,1.0*bio1 - 1.0*bio1_reverse_b18f7
Compartments,"c0, e0"


In [10]:
media = kbase.get_from_ws('Carbon-D-Glucose', ws)
model.medium = media
model.summary()

### Old stuff below ignore

In [163]:
cobra_reaction.genes
for gene in cobra_reaction.genes:
    print(gene, type(gene))

b3177 <class 'cobra.core.gene.Gene'>


In [6]:
model = builder.build()

In [7]:
model

Name,GCF_000005845.2.mdl
Memory address,0x07fb9360b3950
Number of metabolites,1287
Number of reactions,1566
Number of groups,0
Objective expression,1.0*bio1_biomass - 1.0*bio1_biomass_reverse_6e711
Compartments,"c0, e0"


# Run COBRA optimization
Notice the growth rate biomass (*bio1_biomass*) is at 298. Without a specified media all extracellular Exchanges are set to (-1000, 1000)

In [8]:
model.summary()

In [9]:
media_json = kbase.get_object('Carbon-D-Glucose', ws)
media = cobrakbase.core.KBaseBiochemMedia(media_json)
media

In [10]:
media.get_media_constraints()

{'cpd00149_e0': (-100, 100),
 'cpd00099_e0': (-100, 100),
 'cpd00067_e0': (-100, 100),
 'cpd00063_e0': (-100, 100),
 'cpd00058_e0': (-100, 100),
 'cpd00048_e0': (-100, 100),
 'cpd00034_e0': (-100, 100),
 'cpd00030_e0': (-100, 100),
 'cpd00013_e0': (-100, 100),
 'cpd00009_e0': (-100, 100),
 'cpd00001_e0': (-100, 100),
 'cpd00007_e0': (-10, 100),
 'cpd00205_e0': (-100, 100),
 'cpd00254_e0': (-100, 100),
 'cpd00971_e0': (-100, 100),
 'cpd10515_e0': (-100, 100),
 'cpd10516_e0': (-100, 100),
 'cpd11574_e0': (-100, 100),
 'cpd00244_e0': (-100, 100),
 'cpd00027_e0': (-5, 100)}

In [11]:
model = builder.with_media(media).build()

In [12]:
model.summary()

In [13]:
model.genes[:4]

[<Gene b3177 at 0x7fb8f0269f50>,
 <Gene b2947 at 0x7fb8f024ffd0>,
 <Gene b0125 at 0x7fb920616210>,
 <Gene b0775 at 0x7fb8f026e9d0>]

In [14]:
model.genes[0]

Gene identifier,b3177
Name,b3177
Memory address,0x07fb8f0269f50
Functional,True
In 2 reaction(s),"rxn02201_c0, rxn02200_c0"


## Gene Annotation
The model object does not store any annotation for the genes. We can retreive gene data from the Genome object

In [15]:
model.genes[0].annotation

{'sbo': 'SBO:0000243'}

KBase models are attached to a genome object (*genome_ref*)

In [16]:
fbamodel.data['genome_ref']

'45384/3/1'

We get the object (*id/name*) and workspace (*id*) with *get_object_info_from_ref*

In [17]:
ref_info = kbase.get_object_info_from_ref(fbamodel.data['genome_ref'])
print(ref_info.id, ref_info.workspace_id)

GCF_000005845.2.RAST1 filipeliu:narrative_1561151919387


In [18]:
genome_json = kbase.get_object('GCF_000005845.2.RAST1', 'filipeliu:narrative_1561151919387')
genome = cobrakbase.core.KBaseGenome(genome_json)
genome

In [19]:
genome.data['scientific_name']

'Escherichia coli str. K-12 substr. MG1655'

In [20]:
model = builder.with_media(media).with_genome(genome).build()

In [21]:
model.genes[0].annotation

{'sbo': 'SBO:0000243',
 'ncbiprotein': 'NP_417644.4',
 'ncbigene': '947691',
 'ecogene': 'EG50011'}